In [1]:
import numpy as np
from NeuralNetwork import NeuralNetwork
from load import *

In [2]:
X_train, y_train = load_mnist('./data')
X_test, y_test = load_mnist('./data', 't10k')

In [3]:
parameters = {
            'input_size': X_train.shape[1],
            'hidden_size': 80,
            'output_size': 10,
            'learning_rate': 0.001,
            'decay_rate': 0.00001,
            'l2': 0.1,
            'momentum_const': 0.5,
            'minibatch': 500,
            'epochs': 2000,
            'dropout': True,
            'dropout_rate': 0.5,
        }
nn = NeuralNetwork(**parameters)

In [4]:
nn.fit(X_train, y_train, True)
accuracy = nn.accuracy(X_test, y_test)
print("Accuracy:", accuracy)

activations.py:29: RuntimeWarning: overflow encountered in exp
  d = 1.0/(1.0 + np.exp(-w))


Epoch: 50
Loss: 1.29472766508
Training Accuracy: 89.0316666667
Epoch: 100
Loss: 1.14314511025
Training Accuracy: 88.7
Epoch: 150
Loss: 1.41890624583
Training Accuracy: 89.0683333333
Epoch: 200
Loss: 1.26216242925
Training Accuracy: 89.4166666667
Epoch: 250
Loss: 0.957019235733
Training Accuracy: 89.8083333333
Epoch: 300
Loss: 1.31039035251
Training Accuracy: 89.9983333333
Epoch: 350
Loss: 0.872543362134
Training Accuracy: 89.9833333333
Epoch: 400
Loss: 1.00250761248
Training Accuracy: 90.4333333333
Epoch: 450
Loss: 1.01530007718
Training Accuracy: 90.8616666667
Epoch: 500
Loss: 0.961831577673
Training Accuracy: 90.6983333333
Epoch: 550
Loss: 0.658243877392
Training Accuracy: 91.5283333333
Epoch: 600
Loss: 0.65960215238
Training Accuracy: 91.61
Epoch: 650
Loss: 0.968593850422
Training Accuracy: 91.9066666667
Epoch: 700
Loss: 0.656739770596
Training Accuracy: 92.3116666667
Epoch: 750
Loss: 0.718028737072
Training Accuracy: 92.2666666667
Epoch: 800
Loss: 0.748543734806
Training Accuracy: 

In [5]:
accuracy = nn.accuracy(X_test, y_test)
print("Accuracy:", accuracy)

('Accuracy:', 93.4)
